In [1]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import os
import clip
    
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [5]:
model, preprocess = clip.load('ViT-B/32', device)

In [6]:
path_dataset_10 = "/projectnb/ec523kb/projects/chopped/data/food101_10percent"
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
    ])
train_dataset_10 = datasets.ImageFolder(path_dataset_10, transform=transform)
dataloader_10 = DataLoader(train_dataset_10, batch_size=10, shuffle=True)

In [7]:
print(model.visual)

VisionTransformer(
  (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
  (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): Sequential(
      (0): ResidualAttentionBlock(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      )
      (1): ResidualAttentionBlock(
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise

In [8]:
img, label = train_dataset_10[0]

img = transforms.Resize((224, 224))(img)
img = img.unsqueeze(0)
img = img.to(device)
img = img.half()
print(img.shape)

out = model.visual(img)
print(out.shape)
# fig, ax = plt.subplots(1, 3)
# ax[0].imshow(img.permute(1, 2, 0))
# print(label)

/projectnb/ec523kb/students/chrisdc/.conda/envs/chopped/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


torch.Size([1, 3, 224, 224])


/projectnb/ec523kb/students/chrisdc/.conda/envs/chopped/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1699449183005/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


torch.Size([1, 512])


In [11]:
class ClipClassify(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit = model.visual
        self.classification_head = nn.Linear(512, 101)

    def forward(x):
        x = self.vit(x)
        x = self.classification_head(x)
        x = nn.Softmax(x)
        return x

## Fine Tune With Various LLRD

In [10]:
parameters = []
net = ClipClassify().to(device)
print(type(net.vit.transformer.resblocks[11].named_parameters()))
parameters += [{'params': 3}]

layers = [
    
]

layers = [name for name, param in net.named_parameters()]
for layer in layers:
    print(layer)

<class 'generator'>
vit.class_embedding
vit.positional_embedding
vit.proj
vit.conv1.weight
vit.ln_pre.weight
vit.ln_pre.bias
vit.transformer.resblocks.0.attn.in_proj_weight
vit.transformer.resblocks.0.attn.in_proj_bias
vit.transformer.resblocks.0.attn.out_proj.weight
vit.transformer.resblocks.0.attn.out_proj.bias
vit.transformer.resblocks.0.ln_1.weight
vit.transformer.resblocks.0.ln_1.bias
vit.transformer.resblocks.0.mlp.c_fc.weight
vit.transformer.resblocks.0.mlp.c_fc.bias
vit.transformer.resblocks.0.mlp.c_proj.weight
vit.transformer.resblocks.0.mlp.c_proj.bias
vit.transformer.resblocks.0.ln_2.weight
vit.transformer.resblocks.0.ln_2.bias
vit.transformer.resblocks.1.attn.in_proj_weight
vit.transformer.resblocks.1.attn.in_proj_bias
vit.transformer.resblocks.1.attn.out_proj.weight
vit.transformer.resblocks.1.attn.out_proj.bias
vit.transformer.resblocks.1.ln_1.weight
vit.transformer.resblocks.1.ln_1.bias
vit.transformer.resblocks.1.mlp.c_fc.weight
vit.transformer.resblocks.1.mlp.c_fc.bias

In [19]:
clip_classify = ClipClassify().to(device)
print(clip_classify.parameters)

<bound method Module.parameters of ClipClassify(
  (vit): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768,

In [ ]:
criterion = nn.CrossEntropyLoss
clip_classify = ClipClassify().to(device)
optimizer = optim.AdamW(clip_classify.parameters(), )